# Esecuzione

## Impostazione

Per rendere più chiare a me stesso le funzionalità e le loro connessioni, ho fatto una piccola User Story Map che mi aiutasse ad affettare il problema in parti più piccole, "aggredibili" singolarmente.

Ho pertanto isolato le singole funzionalità a disposizione dell'utente e su ciascuna ho immaginato il percorso dell'utente durante l'utilizzo.

Di sequito, dividerò le singole funzionalità in sotto-capitoli per documentare come ho pensato di progettarle, le decisione prese e i razionali sottostanti.

### Decisioni prese

<u>**CLASSE VS FUNZIONI**<u>

Ho riflettuto sul creare una classe che creasse di volta in volta gli ogetti "prodotto". In effetti, ogni prodotto ha un insieme di attributi ben definiti e i metodi potrebbero essere invocati per l'aggiunta dei prodotti o per la vendita.

L'alternativa è procedere alla scrittura di funzioni senza l'utilizzo di classi.

In questo caso, trattandosi di azioni ben precise che dovevano essere fatte dall'utente che, tramite input avrebbe dovuto fornire un insieme di informazioni come prodotto e quantità, mi è sembrato più corretto e diretto procedere con funzioni separate per ogni funzionalità.

Il dover ogni volta creare istanze della classe mi è sembrato un processo eccessivamente macchinoso per questo progetto.

<u>**PERSISTENZA DEI DATI**<u>

Trattandosi di dati strutturati, il file in cui mi è sembrato più sensato andare a censire il magazzino è un csv. In ottica utente, risulta comunque il file più comodo da scambiare o analizzare in caso di reportistica. Ogni colonna sarà un attributo del prodotto e ogni riga un prodotto in magazzino con le conseguenti valorizzazioni.

Stesso discorso dicasi per le vendite.

Con questi 2 file si è in grado di gestire tutte le circostanze del progetto.

<u>**TIPI DI DATI**<u>

Come nell'esercitazione all'interno del modulo sui file, la tipologia di collezione che credo si adatti meglio all'utilizzo di csv, è il dizionario. I nomi delle colonne del csv saranno le chiavi e le valorizzazioni i valori.

## Aggiunta di un nuovo prodotto

Di seguito la descrizione dei passi della funzione e lo User Story Map

![Funzione](./media/def_new_product.jpg)


![USM](./media/USM_new_product.png)



In [66]:
def new_product(prod_name, quantity, price_purch, price_sell):
    """
    Funzione per l'aggiunta di prodotti in magazzino
    """
    import csv
    import os
    """
    Se il file magazzino.csv non esiste quando la funzione viene chiamata, lo crea, scrive i nomi delle colonne
    e inserisce la prima riga con gli argomenti passati.
    Se il file esiste invece verifica se il prodotto ci sia: se si aggiorna la quantità altrimenti inserisce una
    nuova riga
    """ 
    headers_warehouse = ["nome_prodotto", "quantità", "prezzo_acquisto", "prezzo_vendita"]
    
    
    with open("magazzino.csv", "a+", newline="") as wareh_open:
        wareh_open.seek(0)
        if os.path.getsize("./magazzino.csv") == 0:
            wareh_writer = csv.DictWriter(wareh_open, fieldnames = headers_warehouse)
            wareh_writer.writeheader()
            row_1 = {"nome_prodotto":prod_name, "quantità":quantity, "prezzo_acquisto":price_purch, "prezzo_vendita":price_sell}
            wareh_writer.writerow(row_1)
            return print(f"Aggiunto {prod_name} X {quantity}\n")
        
        else:
            wareh_reader = csv.DictReader(wareh_open)
            rows = list(wareh_reader)
            
            update_row = 0
            for row in rows:
                
                if row["nome_prodotto"] == prod_name:
                    row["quantità"] = quantity + int(row["quantità"])
                    update_row += 1
                else:
                    continue
    
    
            """
            update_row == 0 implica che il prodotto aggiunto dall'utente non è stato trovato in magazzino, quindi 
            si aggiunge a rows una nuova riga.
            """    
            if update_row == 0:
                rows.append({"nome_prodotto": prod_name, "quantità": quantity, "prezzo_acquisto": price_purch, "prezzo_vendita": price_sell})
                
    
    
        """
        Il csv magazzino viene aperto nuovamente e il contenuto di rows, ora aggiornato in base alla richiesta
        dell'utente viene sovrascritto al file
        """            
        with open("magazzino.csv", "w", newline="") as wareh_open:
            wareh_writer = csv.DictWriter(wareh_open, fieldnames = headers_warehouse)
            wareh_writer.writeheader()
            wareh_writer.writerows(rows)
            
            if update_row == 0:
                print(f"Aggiunto {prod_name} X {quantity}\n")
                
            else:
                print(f"Quantità di {prod_name} aggiornata\n")
            

## Vendita di un prodotto

Anche in questo caso verrà implementata una funzione che avrà come argomento l'input dell'utente, il quale potrà richiedere la vendita di più prodotti contemporaneamente, ciascuno con la propria quantità.

La funzione farà i seguenti controlli:
* verificare che il prodotto inserito di cui si vuole registrare la vendita esista in magazzino
* verificare che, qualora esista, la quantità presente possa coprire la quantità richiesta per la vendita
* aggiornare la giacenza nel csv del magazzino e registrare la vendita in un csv dedicato.


Se si provasse a vendere un prodotto in una quantità superiore a quella presente in magazzino, il programma si arresterebbe avvisando l'utente dell'errore (maggiori dettagli su questo nella sezione delle decisioni prese).

Anche in caso di prodotto non presente in magazzino, come da specifiche, il programma si arresterebbe restituendo un errore all'utente.

Di seguito User Story Map e flusso logico della funzione

![Flusso logico funzione](./media/def_product_sold.png)

![USM](./media/USM_product_sold.png)


### Decisioni prese

* Dal momento che l'utente può inserire più prodotti e quantità per singola vendita, si è deciso di inserire queste informazioni in un dizionario che sia facilmente utilizzabile all'interno della funzione, poichè ne sarà l'argomento
* Non essendo specificato nella traccia del progetto cosa fare in caso di quantità in magazzino non sufficiente a coprire la vendita, si è deciso di bloccare il programma e restituire un errore parlante. Nonostante non sia una soluzione ideale dal punto di vista dell'utente, riduce la complessità consentendo di non gestire il proseguimento del programma e l'immagazzinamento dei prodotti non gestiti per mostrarli all'utente
* In caso invece di prodotto non trovato in magazzino, si mostrerà all'utente un messaggio di errore come specificato nella traccia. Non essendo però indicato altro, si è deciso di non bloccare il programma e processare la vendita per tutti i prodotti su cui fosse possibile. Alla fine all'utente sarà specificato per quali prodotti la vendita è stata registrata e per quali non, non essendo presenti in magazzino.
* La vendita, per garantire la persistenza dei dati, verrà registrata in un file csv dedicato. Non essendoci specifiche sulla struttura del file o la logica con cui popolarlo, si è deciso di inserire una riga per vendita. Questo, oltre a mantenere le cose semplici, potrebbe essere utile agli utenti in caso di analisi sulle vendite. Nel file sarà anche presente un campo data, utile per fare analisi sulle vendite.


In [110]:
def product_sold(cart):
    
    import csv
    import os
    from datetime import datetime

    prod_selling = list(cart.keys())
    ledger_list = []
    try:
        
        with open("magazzino.csv", newline="") as wareh_open:
                wareh_reader = csv.DictReader(wareh_open)
                rows = list(wareh_reader)
                headers = wareh_reader.fieldnames
    
    except FileNotFoundError:
        return print("Il magazzino è vuoto! Aggiungi qualcosa prima di vendere\n")
            
    with open("vendite.csv", "a+", newline = "") as sales_open:
        sales_headers = ["prodotto", "quantità", "prezzo_acquisto", "prezzo_vendita", "data"]
        
        if os.path.getsize("./vendite.csv") == 0:  
            sales_writer = csv.DictWriter(sales_open, fieldnames = sales_headers)
            sales_writer.writeheader()
            
        sales_reader = csv.DictReader(sales_open)
        

    
    
        """
        Il ciclo controlla che ogni prodotto inserito dall'utente esista in magazzino, Se esiste, verifica che la
        quantità sia sufficiente a coprire la vendita. In caso affermativo su entrambi i controlli si vende diminuendo
        la quantità in magazzino, prendendo la data odierna e creando un dizionario con i dati di vendita che sarà
        poi scritto nel csv vendite. Se la quantità non fosse sufficiente, il programma si ferma restituendo un messaggio.
        """    

        for product in prod_selling:
            for row in rows:
                if product == row["nome_prodotto"]:
                    if cart[product] <= int(row["quantità"]):
                        row["quantità"] = int(row["quantità"]) - cart[product]
                        today = datetime.today().strftime("%d/%m/%Y")
                        ledger_list.append({"prodotto":product, "quantità":cart[product], 
                                            "prezzo_acquisto":row["prezzo_acquisto"], 
                                            "prezzo_vendita":row["prezzo_vendita"], "data":today})
                        
                    else:
                        print(f"\nATTENZIONE! Per il prodotto {product} richiesta {cart[product]} ma presente in magazzino {row["quantità"]}")
                        return                       
                else:
                    continue

    """
    Aggiornamento dei csv magazzino e vendite
    """        
        
    with open("magazzino.csv", "w", newline="") as wareh_open:
        wareh_writer = csv.DictWriter(wareh_open, fieldnames = headers)
        wareh_writer.writeheader()
        wareh_writer.writerows(rows)

        
    with open("vendite.csv", "a", newline = "") as sales_open:
        sales_writer = csv.DictWriter(sales_open, fieldnames = sales_headers)
        sales_writer.writerows(ledger_list)

    """
    Se la vendita è stata effettuata per un numero di prodotti diverso da quelli inseriti dall'utente, vuol dire che
    alcuni di essi non erano presenti a magazzino. La vendita si chiude comunque per quelli per cui è stato possibile.
    Il programma quindi specifica all'utente quali sono stati, di quelli da lui inseriti, i prodotti venduti e quali no.
    """
        
    if len(prod_selling) != len(ledger_list):
        print(f"ATTENZIONE! Vendita non effettuata per questi prodotti perchè non presenti in magazzino: ")
        
        for i in ledger_list:
            prod_selling.remove(i["prodotto"])

        for p in prod_selling:
            print(f"- {p}\n")
        
        print("VENDITA REGISTRATA")
        for i in ledger_list:
            print(f"- {i["quantità"]} X {i["prodotto"]}: €{i["prezzo_vendita"]}")
            
    else:
        print("VENDITA REGISTRATA")
        for i in ledger_list:
            print(f"- {i["quantità"]} X {i["prodotto"]}: €{i["prezzo_vendita"]}")
                        

## Elenca prodotti

Questa funzione non farà altro che stampare a video un sottoinsieme delle colonne del file magazzino.csv.

Le colonne mostrate saranno:
* Prodotto
* Quantità
* Prezzo di vendita

Verrà usata la formattazione per allineare la stampa dei valori e renderla ordinata e leggibile.

In [100]:
def show_warehouse():

    import csv
    
    with open("magazzino.csv", newline="") as wareh_open:
        wareh_reader = csv.DictReader(wareh_open)

        print("{:<30}{:<30}{:<30}".format("PRODOTTO","QUANTITA'","PREZZO"))
        
        for row in wareh_reader:
            print(f'{row["nome_prodotto"]:<30}{row["quantità"]:<30}€{row["prezzo_vendita"]:<30}')

        print("\n")

## Mostra utile lordo e netto

Questa funzione non prende nulla in input, come la funzione per la visualizzazione dei prodotti in magazzino.

Una volta invocata:
* Accederà al file vendite.csv
* Lo leggerà
* Per ogni riga del file sommerà, iterativamente, il prezzo di vendita moltiplicato per la quantità andando così a costruire l'utile lordo
* Per ogni riga del file sommerà, iterativamente, il prezzo di acquisto moltiplicato per la quantità, andando così a costruire i costi totali
* Restituirà, alla fine delle iterazioni
    * l'utile netto come differenza tra utile lordo e totale costi
    * L'utile lordo

In dettaglio dunque:

$Utile\:Lordo = \sum_{i=1}^{n}pv_i*q_i$

$Costi\:Totali = \sum_{i=1}^{n}pa_i*q_i$

$Utile\:Netto = Utile\:Lordo - Costi\:Totali$



In [102]:
def profit():
    import csv
    gross_profit = 0
    total_costs = 0

    with open("vendite.csv", newline="") as sales_open:
        sales_reader = csv.DictReader(sales_open)

        for row in sales_reader:
            gross_profit += int(row["quantità"])*float(row["prezzo_vendita"])
            total_costs += int(row["quantità"])*float(row["prezzo_acquisto"])


    net_profit = gross_profit - total_costs
    
    print(f"Profitti: lordo=€{round(gross_profit,2)} netto=€{round(net_profit,2)}\n")
    

## Aiuto

Questa funzione, una volta ingaggiata, restituirà all'utente la lista dei comandi a sua disposizione.

Ho riflettuto su possibili modo per rendere questa lista dinamica e non dover stampare staticamente ogni singolo comando. Una lista dinamica ha il grande vantaggio che se i comandi fossero stati 100 non avrei dovuto scriverli singolarmente nel codice. Inoltre, se si aggiungessero o eliminassero comandi, bisognerebbe sempre tornare ad aggiornare conseguentemente la funzione.

Purtroppo non sono riuscito a pensare a nessun modo per poter rendere questa lista dinamica e per non spendere ulteriore tempo, ho proceduto con la lista statica.


In [104]:
def help_user():
    print("I comandi disponibili sono i seguenti:")
    print("aggiungi: aggiungi un prodotto al magazzino")
    print("elenca: elenca i prodotto in magazzino")
    print("vendita: registra una vendita effettuata")
    print("profitti: mostra i profitti totali")
    print("aiuto: mostra i possibili comandi")
    print("chiudi: esci dal programma\n")

## Comandi utente

Con questa parte di codice si metteranno insieme tutte le funzioni sviluppate perchè siano richiamabili dall'utente del negozio tramite una finestra di input testuale.

Saranno presenti una serie di if che, in base al comando inserito dall'utente, andranno a richiamare le funzioni passando, laddove serva, gli argomenti necessari al corretto funzionamento.

In questa fase saranno gestiti anche eventuali errori in caso di inserimento errato da parte dell'utente.

I comandi a disposizione dell'utente saranno:
* aggiungi: aggiungi un prodotto al magazzino
* elenca: elenca i prodotto in magazzino
* vendita: registra una vendita effettuata
* profitti: mostra i profitti totali
* aiuto: mostra i possibili comandi
* chiudi: esci dal programma

Dal momento che deve essere previsto un comando "chiudi" che faccia terminare il programma e anche per la vendita di un prodotto. il programma deve continuare a chiedere se l'utente voglia inserire nuovi prodotti, verranno utilizzati dei cicli while che si interromperanno raggiunte le condizioni di stop.

In [114]:
a = True

while a:
    
    command = input("Inserisci un comando: ")

    
    """
    Utilizzato un blocco try/except per gestire i casi in cui si inserisca un nome prodotto non alfabetico,
    o quantità e prezzi non numerici
    """
    try:   

        """
        Blocco di if, uno per ogni possibile comando con richiamo della funzione corrispettiva
        """
        if command == "aggiungi":
            p_add = input("Nome del prodotto: ")
            p_add_no_space = p_add.replace(" ", "")
            assert p_add_no_space.isalpha()
            q_add = int(input("Quantità: "))
            pa = float(input("Prezzo di acquisto: "))
            pv = float(input("Prezzo di vendita: "))
        
            new_product(p_add, q_add, pa, pv)
        
        elif command == "vendita":
            cart = {}
            b=True
            while b:
            
                p_sell = input("Nome del prodotto: ")
                p_sell_no_space = p_sell.replace(" ","")
                assert p_sell_no_space.isalpha()
                q_sell = int(input("Quantità: "))
                cart[p_sell] = q_sell    
                np = input("Aggiungere un altro prodotto? (si/no): ")
                
                if np == "no":
                    b=False
                else:
                    b=True
            
            product_sold(cart)
    
            
        elif command == "elenca":
            show_warehouse()
        
        elif command == "profitti":
            profit()
        
        elif command == "aiuto":
            help_user()
        
        elif command == "chiudi":
            a=False
     
        else:
            print("Comando non valido")
            help_user()

    except ValueError:
        print("Per quantità e prezzo sono ammessi solo valori numerici!\n")
    except AssertionError:
        print("Per il nome del prodotto è ammesso solo testo!\n")

Inserisci un comando:  profitti


Profitti: lordo=€281.4 netto=€130.2



Inserisci un comando:  chiudi


## Fonti

* [CSV File Reading and Writin](https://docs.python.org/3/library/csv.html)
* [Iterable VS Iterator Explained In Python](https://www.youtube.com/watch?v=afPzjnprlsY)
* [Format String Syntax](https://docs.python.org/3/library/string.html#formatstrings)
* [How to print a string at a fixed width?](https://stackoverflow.com/questions/8450472/how-to-print-a-string-at-a-fixed-width)
* [Check if a string is numeric, alphabetic, alphanumeric, or ASCII](https://note.nkmk.me/en/python-str-num-determine/)
* [str.isalpha](https://docs.python.org/3/library/stdtypes.html#str.isalpha)

In [11]:
import csv
import os

headers_warehouse = ["nome_prodotto", "quantità", "prezzo_acquisto", "prezzo_vendita"]
    
with open("test.csv", "a+", newline="") as wareh_open:
    wareh_open.seek(0)
    if os.path.getsize("./test.csv") == 0:
        wareh_writer = csv.DictWriter(wareh_open, fieldnames = headers_warehouse)
        wareh_writer.writeheader()

In [33]:
import os
import csv

os.path.getsize("./magazzino.csv")

with open("magazzino.csv", newline ="") as file_open:
    line_reader = csv.reader(file_open)
    list_reader = list(line_reader)
    print(len(list_reader))

    

1
